In [352]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from PIL import Image

In [353]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

In [354]:
# split train and val data

val_split = .2

val_df = train_df.sample(frac=0.2)
train_df = train_df.drop(val_df.index)

In [357]:
# convert labels to categorical encoding

train_labels = pd.get_dummies(train_df.pop("label")).values
val_labels = pd.get_dummies(val_df.pop("label")).values

In [358]:
# reshape images to 2D pixel grid with 1 channel

def df_to_pixels(df, img_shape):
    pixels = df.apply(lambda x: x.values.reshape(img_shape), axis=1)
    pixels = np.asarray(list(pixels.values)).astype('float32')
    return pixels

In [359]:
img_shape = (28,28,1)

train_pixels = df_to_pixels(train_df, img_shape)
val_pixels = df_to_pixels(val_df, img_shape)
test_pixels = df_to_pixels(test_df, img_shape)

In [385]:
# build tensorflow datasets

batch_size = 128

train_ds = tf.data.Dataset.from_tensor_slices((train_pixels, train_labels)).batch(batch_size)
val_ds = tf.data.Dataset.from_tensor_slices((val_pixels, val_labels)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(test_pixels).batch(1)

### Build convolutional neural net

In [375]:
inputs = keras.Input(shape=(28, 28, 1))

x = layers.Rescaling(scale = 1./255)(inputs)
x = layers.Conv2D(
        filters=6,
        kernel_size=(5,5),
        activation='relu',
        padding='same'
    )(x)
x = layers.AveragePooling2D(strides=2,padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)

outputs = layers.Dense(10, activation='softmax')(x)

In [376]:
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist")

In [377]:
model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer='adam'
)

In [378]:
model.summary()

Model: "mnist"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_51 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 rescaling_49 (Rescaling)    (None, 28, 28, 1)         0         
                                                                 
 conv2d_47 (Conv2D)          (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_30 (Avera  (None, 14, 14, 6)        0         
 gePooling2D)                                                    
                                                                 
 flatten_17 (Flatten)        (None, 1176)              0         
                                                                 
 dense_38 (Dense)            (None, 64)                75328     
                                                             

In [379]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5)

Epoch 1/5
263/263 [==============================] - 5s 16ms/step - loss: 0.5011 - accuracy: 0.8669 - val_loss: 0.1976 - val_accuracy: 0.9433
Epoch 2/5
263/263 [==============================] - 4s 16ms/step - loss: 0.1631 - accuracy: 0.9535 - val_loss: 0.1354 - val_accuracy: 0.9624
Epoch 3/5
263/263 [==============================] - 4s 15ms/step - loss: 0.1204 - accuracy: 0.9647 - val_loss: 0.1186 - val_accuracy: 0.9661
Epoch 4/5
263/263 [==============================] - 4s 15ms/step - loss: 0.1003 - accuracy: 0.9699 - val_loss: 0.1107 - val_accuracy: 0.9668
Epoch 5/5
263/263 [==============================] - 4s 16ms/step - loss: 0.0884 - accuracy: 0.9733 - val_loss: 0.1082 - val_accuracy: 0.9682


In [387]:
predictions = model.predict(test_ds)

28000/28000 [==============================] - 49s 2ms/step


In [389]:
numerical_predictions = np.argmax(predictions, axis=1)

In [396]:
output_df = pd.read_csv("../submissions/sample_submission.csv")
output_df["Label"] = numerical_predictions

In [400]:
output_df.to_csv("../submissions/simple_convnet.csv", index=False)